In [1]:
import numpy as np
import tensorflow as tf
from collections import OrderedDict #순서까지 기억하는 사전형
import os, random
import pickle
# cifar 10 
#wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
#tar xzf cifar-10-python.tar.gz
# resnet
#http://laonple.blog.me/220761052425
#http://laonple.blog.me/220770760226

#TFRECODE ENCODE
#https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/how_tos/reading_data/convert_to_records.py

#TFRECODE DECODE
#https://github.com/tensorflow/tensorflow/blob/r1.1/tensorflow/examples/how_tos/reading_data/fully_connected_reader.py
#supervisor

In [2]:
# 파라미터들
input_size = 32
data_path = 'cifar10/cifar-10-batches-py/'

#train&test batch
train_file=['data_batch_1','data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file=['cifar10/cifar-10-batches-py/test_batch']


list_class=['airplane', 'automobile', 'birds', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
n_side = 32
n_size = n_side * n_side
n_channel = 3

n_class = len(list_class)
n_layer = 3 #simple resnet

n_batch = 100

#input & output 
x = tf.placeholder("float", [n_batch, n_size * n_channel]) #batch x image size
y = tf.placeholder("float", [n_batch, n_class]) #batch x class

#weight&bias
stddev = 0.1
weights = {
    'c': tf.Variable(tf.random_normal([3, 3, n_channel, 16], stddev=stddev)),
    'c1_1': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_2': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_3': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c1_4': tf.Variable(tf.random_normal([3, 3, 16, 16], stddev=stddev)),
    'c2_1': tf.Variable(tf.random_normal([3, 3, 16, 32], stddev=stddev)),
    'c2_2': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c2_3': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c2_4': tf.Variable(tf.random_normal([3, 3, 32, 32], stddev=stddev)),
    'c3_1': tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=stddev)),
    'c3_2': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'c3_3': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'c3_4': tf.Variable(tf.random_normal([3, 3, 64, 64], stddev=stddev)),
    'd1' :tf.Variable(tf.random_normal([16*16*64, n_class], stddev=stddev)),
    #'d2' :tf.Variable(tf.random_normal([1000, n_class], stddev=stddev))
}
biases = {
    'c': tf.Variable(tf.random_normal([16], stddev=stddev)), 
    'c1_1': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_2': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_3': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c1_4': tf.Variable(tf.random_normal([16], stddev=stddev)),
    'c2_1': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_2': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_3': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c2_4': tf.Variable(tf.random_normal([32], stddev=stddev)),
    'c3_1': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_2': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_3': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'c3_4': tf.Variable(tf.random_normal([64], stddev=stddev)),
    'd1': tf.Variable(tf.random_normal([n_class], stddev=stddev)),
    #'d2': tf.Variable(tf.random_normal([n_class], stddev=stddev))
}

#시작은 맥스풀
#끝은 에버리지 풀 
# 3x3 conv 2개마다 2개 이전의 결과를 더해줌 
#마지막은 Fc1000

In [3]:
#model
def ResNet(img_width, img_height, img_channel, _x, _w, _b, scope='Simple_ResNet'):
    _names = OrderedDict()

    #reshape
    _x_r = tf.reshape(_x, shape=[-1, img_width, img_height, img_channel])
    
    with tf.variable_scope(scope):
        #max pool
        _pool = tf.nn.max_pool(_x_r, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
        _names['pool'] = _pool
        
        #convolution
        _conv = tf.nn.conv2d(_pool, _w['c'], strides=[1,1,1,1], padding='SAME')
        _conv = tf.nn.bias_add(_conv, _b['c'])
        _conv = tf.nn.relu(_conv)
        _names['conv'] = _conv
        
        #residual conv - 1
        conv1_1 = tf.nn.conv2d(_conv, _w['c1_1'], strides=[1,1,1,1], padding='SAME')
        conv1_1 = tf.nn.bias_add(conv1_1, _b['c1_1'])
        conv1_1 = tf.nn.relu(conv1_1)
        _names['conv1_1'] = conv1_1
        
        conv1_2 = tf.nn.conv2d(conv1_1, _w['c1_2'], strides=[1,1,1,1], padding='SAME')
        conv1_2 = tf.nn.bias_add(conv1_2, _b['c1_2'])
        conv1_2 = tf.add(conv1_2 , _conv) #residual? 
        conv1_2 = tf.nn.relu(conv1_2)
        _names['conv1_2'] = conv1_2
        
        conv1_3 = tf.nn.conv2d(conv1_2, _w['c1_3'], strides=[1,1,1,1], padding='SAME')
        conv1_3 = tf.nn.bias_add(conv1_3, _b['c1_3'])
        conv1_3 = tf.nn.relu(conv1_3)
        _names['conv1_3'] = conv1_3
        
        conv1_4 = tf.nn.conv2d(conv1_3, _w['c1_4'], strides=[1,1,1,1], padding='SAME')
        conv1_4 = tf.nn.bias_add(conv1_4, _b['c1_4'])
        conv1_4 = tf.add(conv1_4 , conv1_2) #residual? 
        conv1_4 = tf.nn.relu(conv1_4)
        _names['conv1_4'] = conv1_4
        
        #residual conv - 2
        conv2_1 = tf.nn.conv2d(conv1_4, _w['c2_1'], strides=[1,1,1,1], padding='SAME')
        conv2_1 = tf.nn.bias_add(conv2_1, _b['c2_1'])
        conv2_1 = tf.nn.relu(conv2_1)
        _names['conv2_1'] = conv2_1
        
        conv2_2 = tf.nn.conv2d(conv2_1, _w['c2_2'], strides=[1,1,1,1], padding='SAME')
        conv2_2 = tf.nn.bias_add(conv2_2, _b['c2_2'])
        #conv2_2 = tf.add(conv2_2, conv1_4)
        conv2_2 = tf.add(conv2_2, conv2_1)
        conv2_2 = tf.nn.relu(conv2_2)
        _names['conv2_2'] = conv2_2
        
        conv2_3 = tf.nn.conv2d(conv2_2, _w['c2_3'], strides=[1,1,1,1], padding='SAME')
        conv2_3 = tf.nn.bias_add(conv2_3, _b['c2_3'])
        conv2_3 = tf.nn.relu(conv2_3)
        _names['conv2_3'] = conv2_3
        
        conv2_4 = tf.nn.conv2d(conv2_3, _w['c2_4'], strides=[1,1,1,1], padding='SAME')
        conv2_4 = tf.nn.bias_add(conv2_4, _b['c2_4'])
        conv2_4 = tf.add(conv2_4, conv2_2)
        conv2_4 = tf.nn.relu(conv2_4)
        _names['conv2_4'] = conv2_4
        
        #residual conv - 3
        conv3_1 = tf.nn.conv2d(conv2_4, _w['c3_1'], strides=[1,1,1,1], padding='SAME')
        conv3_1 = tf.nn.bias_add(conv3_1, _b['c3_1'])
        conv3_1 = tf.nn.relu(conv3_1)
        _names['conv3_1'] = conv3_1
        
        conv3_2 = tf.nn.conv2d(conv3_1, _w['c3_2'], strides=[1,1,1,1], padding='SAME')
        conv3_2 = tf.nn.bias_add(conv3_2, _b['c3_2'])
        #conv3_2 = tf.add(conv3_2, conv2_4)
        conv3_2 = tf.add(conv3_2, conv3_1)
        conv3_2 = tf.nn.relu(conv3_2)
        _names['conv3_2'] = conv3_2
        
        conv3_3 = tf.nn.conv2d(conv3_2, _w['c3_3'], strides=[1,1,1,1], padding='SAME')
        conv3_3 = tf.nn.bias_add(conv3_3, _b['c3_3'])
        conv3_2 = tf.nn.relu(conv3_3)
        _names['conv3_3'] = conv3_3
        
        conv3_4 = tf.nn.conv2d(conv3_3, _w['c3_4'], strides=[1,1,1,1], padding='SAME')
        conv3_4 = tf.nn.bias_add(conv3_4, _b['c3_4'])
        conv3_4 = tf.add(conv3_4, conv3_2)
        conv3_4 = tf.nn.relu(conv3_4)
        _names['conv3_4'] = conv3_4
        
        
        
        #vectorize
        _dense = tf.reshape(conv3_4, [-1, _w['d1'].get_shape().as_list()[0]])
        _names['dense'] = _dense
        
        #dense
        _logit = tf.add(tf.matmul(_dense, _w['d1']), _b['d1'])
        _names['logit'] = _logit
                
    return _names        

In [4]:
#prediction
out = ResNet(n_side, n_side, n_channel, x, weights, biases, 'Simple_ResNet')
print(out)

OrderedDict([('pool', <tf.Tensor 'Simple_ResNet/MaxPool:0' shape=(100, 16, 16, 3) dtype=float32>), ('conv', <tf.Tensor 'Simple_ResNet/Relu:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_1', <tf.Tensor 'Simple_ResNet/Relu_1:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_2', <tf.Tensor 'Simple_ResNet/Relu_2:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_3', <tf.Tensor 'Simple_ResNet/Relu_3:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_4', <tf.Tensor 'Simple_ResNet/Relu_4:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv2_1', <tf.Tensor 'Simple_ResNet/Relu_5:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv2_2', <tf.Tensor 'Simple_ResNet/Relu_6:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv2_3', <tf.Tensor 'Simple_ResNet/Relu_7:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv2_4', <tf.Tensor 'Simple_ResNet/Relu_8:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv3_1', <tf.Tensor 'Simple_ResNet/Relu_9:0' shape=(100, 16, 16, 64) dtype=float32>), ('conv3_2', <tf.

In [5]:
#cost function & loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out['logit']))

In [6]:
#optimizer
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
corr = tf.equal(tf.argmax(out['logit'], 1), tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

In [7]:
#initializer
init = tf.global_variables_initializer()
print ("FUNCTIONS READY")

FUNCTIONS READY


In [8]:
print(y)
print(out['dense'])
print(out)

Tensor("Placeholder_1:0", shape=(100, 10), dtype=float32)
Tensor("Simple_ResNet/Reshape:0", shape=(100, 16384), dtype=float32)
OrderedDict([('pool', <tf.Tensor 'Simple_ResNet/MaxPool:0' shape=(100, 16, 16, 3) dtype=float32>), ('conv', <tf.Tensor 'Simple_ResNet/Relu:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_1', <tf.Tensor 'Simple_ResNet/Relu_1:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_2', <tf.Tensor 'Simple_ResNet/Relu_2:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_3', <tf.Tensor 'Simple_ResNet/Relu_3:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv1_4', <tf.Tensor 'Simple_ResNet/Relu_4:0' shape=(100, 16, 16, 16) dtype=float32>), ('conv2_1', <tf.Tensor 'Simple_ResNet/Relu_5:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv2_2', <tf.Tensor 'Simple_ResNet/Relu_6:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv2_3', <tf.Tensor 'Simple_ResNet/Relu_7:0' shape=(100, 16, 16, 32) dtype=float32>), ('conv2_4', <tf.Tensor 'Simple_ResNet/Relu_8:0' shape=(100, 16, 16,

In [9]:
#save
savedir = "nets/cnn_cifar10_resnet/"
saver = tf.train.Saver(max_to_keep=3)
save_step = 4
if not os.path.exists(savedir):
 os.makedirs(savedir)
print ("SAVER READY")

SAVER READY


In [10]:
def load_npz(file):
 
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
        return dict
    
## Make batch_iterator for npz data
def make_batch_index(_data, batch_size=128, allow_small_batch=True):
    num_images = len(_data[b'data'])
    start_idx = list(range(0, num_images, batch_size))
    end_idx = list(range(batch_size, num_images + 1, batch_size))
    if allow_small_batch and end_idx[-1] < num_images :
        start_idx.append(end_idx[-1])
        end_idx.append(num_images)
    return zip(start_idx, end_idx)

def one_hot_encode(x, numclass):
    return np.eye(numclass)[x]

In [11]:
training_epochs = 1000
batch_size = 100
disp_each = 10
disp_batch =100
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)

In [12]:
load_testdata=load_npz(test_file[0])

In [13]:
#training
for epoch in range(training_epochs):
    
    sample_list_new = random.sample(train_file, len(train_file))
    nNumBatch = 0
    AvgBatchCost = 0
    
    for index, file_name in enumerate(sample_list_new):
        file_name = data_path+file_name
        load_data = load_npz(file_name)
        batch_index = make_batch_index(load_data, batch_size, True)
        
        for start, end in batch_index:
            
            batch_xs = load_data[b'data'][start:end]
            batch_ys = one_hot_encode(load_data[b'labels'][start:end], n_class)
            #print(batch_ys)
            #print(batch_xs, shape)
            feeds = {x: batch_xs, y: batch_ys}
            sess.run(optm, feed_dict = feeds)
            nNumBatch+=1
            tmp_cost, _ = sess.run([cost, optm], feed_dict = feeds)
            AvgBatchCost += tmp_cost
        
        if nNumBatch % disp_batch == 0:
                train_acc = sess.run(accr, feed_dict = feeds)
                print('\t[%d nNumBatch] train_cost = %g, acc = %g'%(nNumBatch, AvgBatchCost/nNumBatch, train_acc))
                
    if epoch % disp_each == 0:
        num_batch = 0
        test_acc = 0
        avg_acc = 0
        load_test_data = load_npz(test_file[0])
        testdatalabel = one_hot_encode(load_testdata[b'labels'], n_class)
        test_batch_index = make_batch_index(load_data, batch_size, True)
                
        for start, end in test_batch_index:
            num_batch += 1
            batch_xs = load_test_data[b'data'][start:end]
            batch_ys = one_hot_encode(load_testdata[b'labels'][start:end],n_class)
            test_acc = sess.run(accr, feed_dict={x:batch_xs,y:batch_ys})
            avg_acc+=test_acc
        print('[%d epoch]test acc=%g'%(epoch, avg_acc/num_batch))

	[100 nNumBatch] train_cost = 716.306, acc = 0.17
	[200 nNumBatch] train_cost = 364.072, acc = 0.15
	[300 nNumBatch] train_cost = 243.732, acc = 0.14
	[400 nNumBatch] train_cost = 183.4, acc = 0.13
	[500 nNumBatch] train_cost = 147.185, acc = 0.15
[0 epoch]test acc=0.1436
	[100 nNumBatch] train_cost = 2.2981, acc = 0.11
	[200 nNumBatch] train_cost = 2.29094, acc = 0.12
	[300 nNumBatch] train_cost = 2.28888, acc = 0.12
	[400 nNumBatch] train_cost = 2.2879, acc = 0.16
	[500 nNumBatch] train_cost = 2.28461, acc = 0.14
	[100 nNumBatch] train_cost = 2.25943, acc = 0.12
	[200 nNumBatch] train_cost = 2.25925, acc = 0.14
	[300 nNumBatch] train_cost = 2.25472, acc = 0.17
	[400 nNumBatch] train_cost = 2.25194, acc = 0.16
	[500 nNumBatch] train_cost = 2.24828, acc = 0.13
	[100 nNumBatch] train_cost = 2.23911, acc = 0.14
	[200 nNumBatch] train_cost = 2.23468, acc = 0.16
	[300 nNumBatch] train_cost = 2.2305, acc = 0.16
	[400 nNumBatch] train_cost = 2.22915, acc = 0.13
	[500 nNumBatch] train_cost = 

	[400 nNumBatch] train_cost = 1.47024, acc = 0.51
	[500 nNumBatch] train_cost = 1.46886, acc = 0.44
	[100 nNumBatch] train_cost = 1.41152, acc = 0.47
	[200 nNumBatch] train_cost = 1.44785, acc = 0.53
	[300 nNumBatch] train_cost = 1.45739, acc = 0.53
	[400 nNumBatch] train_cost = 1.46426, acc = 0.56
	[500 nNumBatch] train_cost = 1.46233, acc = 0.54
	[100 nNumBatch] train_cost = 1.46068, acc = 0.57
	[200 nNumBatch] train_cost = 1.47231, acc = 0.53
	[300 nNumBatch] train_cost = 1.46681, acc = 0.53
	[400 nNumBatch] train_cost = 1.47149, acc = 0.52
	[500 nNumBatch] train_cost = 1.46949, acc = 0.39
	[100 nNumBatch] train_cost = 1.40204, acc = 0.5
	[200 nNumBatch] train_cost = 1.4248, acc = 0.53
	[300 nNumBatch] train_cost = 1.4382, acc = 0.54
	[400 nNumBatch] train_cost = 1.44629, acc = 0.5
	[500 nNumBatch] train_cost = 1.45153, acc = 0.56
	[100 nNumBatch] train_cost = 1.4336, acc = 0.48
	[200 nNumBatch] train_cost = 1.44242, acc = 0.5
	[300 nNumBatch] train_cost = 1.44482, acc = 0.47
	[400 

	[200 nNumBatch] train_cost = 1.35347, acc = 0.61
	[300 nNumBatch] train_cost = 1.34689, acc = 0.51
	[400 nNumBatch] train_cost = 1.36071, acc = 0.53
	[500 nNumBatch] train_cost = 1.36834, acc = 0.54
	[100 nNumBatch] train_cost = 1.32877, acc = 0.5
	[200 nNumBatch] train_cost = 1.36048, acc = 0.59
	[300 nNumBatch] train_cost = 1.36577, acc = 0.63
	[400 nNumBatch] train_cost = 1.37872, acc = 0.57
	[500 nNumBatch] train_cost = 1.38372, acc = 0.52
	[100 nNumBatch] train_cost = 1.34817, acc = 0.56
	[200 nNumBatch] train_cost = 1.35689, acc = 0.52
	[300 nNumBatch] train_cost = 1.35677, acc = 0.48
	[400 nNumBatch] train_cost = 1.3658, acc = 0.54
	[500 nNumBatch] train_cost = 1.37439, acc = 0.58
	[100 nNumBatch] train_cost = 1.35895, acc = 0.53
	[200 nNumBatch] train_cost = 1.35041, acc = 0.49
	[300 nNumBatch] train_cost = 1.36376, acc = 0.6
	[400 nNumBatch] train_cost = 1.36768, acc = 0.6
	[500 nNumBatch] train_cost = 1.3695, acc = 0.56
	[100 nNumBatch] train_cost = 1.33057, acc = 0.62
	[200

	[500 nNumBatch] train_cost = 1.33091, acc = 0.54
	[100 nNumBatch] train_cost = 1.305, acc = 0.61
	[200 nNumBatch] train_cost = 1.30398, acc = 0.53
	[300 nNumBatch] train_cost = 1.33338, acc = 0.62
	[400 nNumBatch] train_cost = 1.32511, acc = 0.63
	[500 nNumBatch] train_cost = 1.32653, acc = 0.62
	[100 nNumBatch] train_cost = 1.23603, acc = 0.64
	[200 nNumBatch] train_cost = 1.26454, acc = 0.6
	[300 nNumBatch] train_cost = 1.27883, acc = 0.67
	[400 nNumBatch] train_cost = 1.28987, acc = 0.53
	[500 nNumBatch] train_cost = 1.27573, acc = 0.53
	[100 nNumBatch] train_cost = 1.32277, acc = 0.58
	[200 nNumBatch] train_cost = 1.2265, acc = 0.62
	[300 nNumBatch] train_cost = 1.25001, acc = 0.61
	[400 nNumBatch] train_cost = 1.26167, acc = 0.6
	[500 nNumBatch] train_cost = 1.26787, acc = 0.65
[100 epoch]test acc=0.4569
	[100 nNumBatch] train_cost = 1.25483, acc = 0.66
	[200 nNumBatch] train_cost = 1.24894, acc = 0.66
	[300 nNumBatch] train_cost = 1.24074, acc = 0.58
	[400 nNumBatch] train_cost 

	[300 nNumBatch] train_cost = 1.21607, acc = 0.66
	[400 nNumBatch] train_cost = 1.22468, acc = 0.7
	[500 nNumBatch] train_cost = 1.22045, acc = 0.66
[130 epoch]test acc=0.4082
	[100 nNumBatch] train_cost = 1.25739, acc = 0.67
	[200 nNumBatch] train_cost = 1.2108, acc = 0.64
	[300 nNumBatch] train_cost = 1.23204, acc = 0.75
	[400 nNumBatch] train_cost = 1.2356, acc = 0.65
	[500 nNumBatch] train_cost = 1.2471, acc = 0.59
	[100 nNumBatch] train_cost = 1.14182, acc = 0.62
	[200 nNumBatch] train_cost = 1.08359, acc = 0.87
	[300 nNumBatch] train_cost = 1.15601, acc = 0.72
	[400 nNumBatch] train_cost = 1.15802, acc = 0.75
	[500 nNumBatch] train_cost = 1.18059, acc = 0.61
	[100 nNumBatch] train_cost = 1.16411, acc = 0.69
	[200 nNumBatch] train_cost = 1.18732, acc = 0.6
	[300 nNumBatch] train_cost = 1.20627, acc = 0.64
	[400 nNumBatch] train_cost = 1.17774, acc = 0.88
	[500 nNumBatch] train_cost = 1.20594, acc = 0.57
	[100 nNumBatch] train_cost = 1.2132, acc = 0.73
	[200 nNumBatch] train_cost =

...

	[200 nNumBatch] train_cost = 0.439333, acc = 0.97
	[300 nNumBatch] train_cost = 0.434815, acc = 0.86
	[400 nNumBatch] train_cost = 0.405183, acc = 0.97
	[500 nNumBatch] train_cost = 0.416985, acc = 0.85
	[100 nNumBatch] train_cost = 0.305506, acc = 0.95
	[200 nNumBatch] train_cost = 0.297631, acc = 0.96
	[300 nNumBatch] train_cost = 0.282404, acc = 0.98
	[400 nNumBatch] train_cost = 0.306215, acc = 0.98
	[500 nNumBatch] train_cost = 0.316837, acc = 0.97
[870 epoch]test acc=0.3761
	[100 nNumBatch] train_cost = 0.346293, acc = 0.94
	[200 nNumBatch] train_cost = 0.297657, acc = 0.98
	[300 nNumBatch] train_cost = 0.270732, acc = 0.99
	[400 nNumBatch] train_cost = 0.568649, acc = 0.48
	[500 nNumBatch] train_cost = 0.705251, acc = 0.51
	[100 nNumBatch] train_cost = 0.994548, acc = 0.89
	[200 nNumBatch] train_cost = 0.801923, acc = 0.93
	[300 nNumBatch] train_cost = 0.695426, acc = 0.94
	[400 nNumBatch] train_cost = 0.624179, acc = 0.92
	[500 nNumBatch] train_cost = 0.589059, acc = 0.98
	[10

	[200 nNumBatch] train_cost = 1.34004, acc = 0.65
	[300 nNumBatch] train_cost = 1.31175, acc = 0.52
	[400 nNumBatch] train_cost = 1.28259, acc = 0.73
	[500 nNumBatch] train_cost = 1.26341, acc = 0.72
	[100 nNumBatch] train_cost = 1.08901, acc = 0.46
	[200 nNumBatch] train_cost = 1.06517, acc = 0.76
	[300 nNumBatch] train_cost = 1.01944, acc = 0.85
	[400 nNumBatch] train_cost = 0.990627, acc = 0.78
	[500 nNumBatch] train_cost = 0.955737, acc = 0.72
	[100 nNumBatch] train_cost = 0.5169, acc = 0.88
	[200 nNumBatch] train_cost = 0.672851, acc = 0.92
	[300 nNumBatch] train_cost = 0.65276, acc = 0.8
	[400 nNumBatch] train_cost = 0.627338, acc = 0.86
	[500 nNumBatch] train_cost = 0.595683, acc = 0.95
	[100 nNumBatch] train_cost = 0.943082, acc = 0.78
	[200 nNumBatch] train_cost = 0.831908, acc = 0.94
	[300 nNumBatch] train_cost = 0.866662, acc = 0.56
	[400 nNumBatch] train_cost = 0.963196, acc = 0.64
	[500 nNumBatch] train_cost = 0.980365, acc = 0.7
	[100 nNumBatch] train_cost = 1.34842, acc 

	[200 nNumBatch] train_cost = 0.279763, acc = 0.94
	[300 nNumBatch] train_cost = 0.270677, acc = 0.91
	[400 nNumBatch] train_cost = 0.2831, acc = 0.98
	[500 nNumBatch] train_cost = 0.266893, acc = 0.99
	[100 nNumBatch] train_cost = 0.275426, acc = 0.94
	[200 nNumBatch] train_cost = 0.227546, acc = 0.98
	[300 nNumBatch] train_cost = 0.258561, acc = 0.95
	[400 nNumBatch] train_cost = 0.25735, acc = 0.97
	[500 nNumBatch] train_cost = 0.297153, acc = 0.93
	[100 nNumBatch] train_cost = 0.289729, acc = 0.99
	[200 nNumBatch] train_cost = 0.430621, acc = 0.94
	[300 nNumBatch] train_cost = 0.40311, acc = 0.96
	[400 nNumBatch] train_cost = 0.382718, acc = 0.95
	[500 nNumBatch] train_cost = 0.375346, acc = 0.93
	[100 nNumBatch] train_cost = 0.238515, acc = 0.97
	[200 nNumBatch] train_cost = 0.235974, acc = 0.99
	[300 nNumBatch] train_cost = 0.252504, acc = 0.96
	[400 nNumBatch] train_cost = 0.262247, acc = 0.98
	[500 nNumBatch] train_cost = 0.269602, acc = 0.98
	[100 nNumBatch] train_cost = 0.634

	[200 nNumBatch] train_cost = 0.244806, acc = 0.98
	[300 nNumBatch] train_cost = 0.287489, acc = 0.91
	[400 nNumBatch] train_cost = 0.284187, acc = 0.93
	[500 nNumBatch] train_cost = 0.271303, acc = 1
	[100 nNumBatch] train_cost = 0.2024, acc = 0.99
	[200 nNumBatch] train_cost = 0.235134, acc = 0.96
	[300 nNumBatch] train_cost = 0.242315, acc = 0.98
	[400 nNumBatch] train_cost = 0.250816, acc = 0.93
	[500 nNumBatch] train_cost = 0.253766, acc = 0.98
	[100 nNumBatch] train_cost = 0.177082, acc = 0.99
	[200 nNumBatch] train_cost = 0.23526, acc = 1
	[300 nNumBatch] train_cost = 0.291022, acc = 0.9
	[400 nNumBatch] train_cost = 0.301034, acc = 0.97
	[500 nNumBatch] train_cost = 0.293807, acc = 0.96
	[100 nNumBatch] train_cost = 0.19761, acc = 0.98
	[200 nNumBatch] train_cost = 0.182451, acc = 0.97
	[300 nNumBatch] train_cost = 0.185429, acc = 1
	[400 nNumBatch] train_cost = 0.205184, acc = 1
	[500 nNumBatch] train_cost = 0.229546, acc = 0.98
	[100 nNumBatch] train_cost = 0.157606, acc = 0.

	[300 nNumBatch] train_cost = 0.241093, acc = 0.96
	[400 nNumBatch] train_cost = 0.243533, acc = 0.98
	[500 nNumBatch] train_cost = 0.24766, acc = 0.99
	[100 nNumBatch] train_cost = 0.224069, acc = 0.97
	[200 nNumBatch] train_cost = 0.208749, acc = 0.97
	[300 nNumBatch] train_cost = 0.209494, acc = 0.98
	[400 nNumBatch] train_cost = 0.222924, acc = 0.96
	[500 nNumBatch] train_cost = 0.219837, acc = 1
	[100 nNumBatch] train_cost = 0.220711, acc = 0.98
	[200 nNumBatch] train_cost = 0.274576, acc = 0.94
	[300 nNumBatch] train_cost = 0.355536, acc = 0.94
	[400 nNumBatch] train_cost = 0.341068, acc = 0.95
	[500 nNumBatch] train_cost = 0.342915, acc = 0.96
